In [1]:
import warnings
warnings.filterwarnings('ignore')

## Loading and preprocessing dataset from Moabb by Braindecode

In [3]:
from braindecode.datasets.moabb import MOABBDataset
from braindecode.datautil.preprocess import (exponential_moving_standardize, preprocess, Preprocessor)
from braindecode.datautil.windowers import create_windows_from_events
import EEGReptile as er

ModuleNotFoundError: No module named 'EEGReptile'

In [ ]:
subject_ids = [1, 2, 3, 4, 5, 6, 7, 8, 9]
ds = MOABBDataset(dataset_name="BNCI2014001", subject_ids=subject_ids)   # loading dataset from MOABB
low_cut = 4.
high_cut = 38.
factor_new = 1e-3
init_block_size = 1000
preprocessors = [
    Preprocessor('pick_types', eeg=True, meg=False, stim=False),
    Preprocessor(lambda x: x*1e3),
    Preprocessor('filter', l_freq=low_cut, h_freq=high_cut),
    Preprocessor(exponential_moving_standardize, factor_new=factor_new, init_block_size=init_block_size)
]
preprocess(ds, preprocessors)
trial_start_ofset = -0.5
sfreq = ds.datasets[0].raw.info['sfreq']
assert all([d.raw.info['sfreq'] == sfreq for d in ds.datasets])
trial_start_ofset_samples = int(trial_start_ofset * sfreq)
w_ds = create_windows_from_events(
    ds,
    trial_start_offset_samples=trial_start_ofset_samples,
    trial_stop_offset_samples=0,
    window_size_samples=625,
    window_stride_samples=500,
    preload=False,
    drop_last_window=True,
    drop_bad_windows=True
)
dsub = w_ds.split("subject")

## Create Reptile-EEG dataset structure
- dataset_name - name of the dataset for access it and its folder name
- description - some str information to store description of loaded data

In [ ]:
dataset = er.MetaDataset(dataset_name='BCI_IV',
                         description='BCI IV 2a dataset filtered 4 - 38hz and standardaized')  # Creating dataset for EEG-reptile lib

## Loading data from Braindecode windows dataset to Reptile-EEG dataset
dataset.add_subject_from_xy - function for uploading X - EEG data, Y - class labels in EEG-Reptile dataset.
- subject_id - int
- x - X data shaped **[epochs, channels, times]**
- y - integer class labels **[epochs]**
- test_size - float, specifies size of test set (0 - 1)

In [ ]:
for sub in subject_ids:
    X = []
    Y = []
    Window = []
    sds = dsub[str(sub)].split("session")
    for x, y, window in sds["0train"]:
        X.append(x)
        Y.append(y)
        Window.append(window)
    for x, y, window in sds["1test"]:
        X.append(x)
        Y.append(y)
        Window.append(window)
    dataset.add_subject_from_xy(subject_id=sub, x=X, y=Y, test_size=0.2)